### Import necessary libraries


In [216]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
import pickle

### Load the datasets into a pandas DataFrame and Check for Basics stuffs

In [217]:
books = pd.read_csv(r'D:\ML Projects\Recommender System for Books\Data\Books.csv')
users = pd.read_csv(r'D:\ML Projects\Recommender System for Books\Data\Users.csv')
ratings = pd.read_csv(r'D:\ML Projects\Recommender System for Books\Data\Ratings.csv')

C:\Users\RAVI TEJ\AppData\Local\Temp\ipykernel_13336\2719581062.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv(r'D:\ML Projects\Recommender System for Books\Data\Books.csv')


In [218]:
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [219]:
# Display the first few rows of the 'Image-URL-M' column in the books dataset to inspect the URLs
books['Image-URL-M'].head()

0    http://images.amazon.com/images/P/0195153448.0...
1    http://images.amazon.com/images/P/0002005018.0...
2    http://images.amazon.com/images/P/0060973129.0...
3    http://images.amazon.com/images/P/0374157065.0...
4    http://images.amazon.com/images/P/0393045218.0...
Name: Image-URL-M, dtype: object

In [220]:
# Replace 'http' with 'https' in the 'Image-URL-M' column
books['Image-URL-M'] = books['Image-URL-M'].str.replace('http://', 'https://')

# Verify the changes
books['Image-URL-M'].head()

0    https://images.amazon.com/images/P/0195153448....
1    https://images.amazon.com/images/P/0002005018....
2    https://images.amazon.com/images/P/0060973129....
3    https://images.amazon.com/images/P/0374157065....
4    https://images.amazon.com/images/P/0393045218....
Name: Image-URL-M, dtype: object

In [221]:
users.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [222]:
ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [223]:
# Print the shape 
print(books.shape)
print(ratings.shape)
print(users.shape)

(271360, 8)
(1149780, 3)
(278858, 3)


In [224]:
# Display concise summary information about the books dataset including the data types
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271360 entries, 0 to 271359
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ISBN                 271360 non-null  object
 1   Book-Title           271360 non-null  object
 2   Book-Author          271358 non-null  object
 3   Year-Of-Publication  271360 non-null  object
 4   Publisher            271358 non-null  object
 5   Image-URL-S          271360 non-null  object
 6   Image-URL-M          271360 non-null  object
 7   Image-URL-L          271357 non-null  object
dtypes: object(8)
memory usage: 16.6+ MB


In [225]:
# Display concise summary information about the ratings dataset including the data types
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149780 entries, 0 to 1149779
Data columns (total 3 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   User-ID      1149780 non-null  int64 
 1   ISBN         1149780 non-null  object
 2   Book-Rating  1149780 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 26.3+ MB


In [226]:
# Display concise summary information about the users dataset including the data types
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278858 entries, 0 to 278857
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   User-ID   278858 non-null  int64  
 1   Location  278858 non-null  object 
 2   Age       168096 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 6.4+ MB


In [227]:
# Checking of null values
books.isnull().sum()

ISBN                   0
Book-Title             0
Book-Author            2
Year-Of-Publication    0
Publisher              2
Image-URL-S            0
Image-URL-M            0
Image-URL-L            3
dtype: int64

In [228]:
# Checking of null values
users.isnull().sum()

User-ID          0
Location         0
Age         110762
dtype: int64

In [229]:
# Checking of null values
ratings.isnull().sum()

User-ID        0
ISBN           0
Book-Rating    0
dtype: int64

In [230]:
# Checking for duplicated rows
books.duplicated().sum()

0

In [231]:
# Checking for duplicated rows
ratings.duplicated().sum()

0

In [232]:
# Checking for duplicated rows
users.duplicated().sum()

0

In [233]:
# Checking for book cover image url that it is cosistent to fetched out or not
books['Image-URL-M'][1]

'https://images.amazon.com/images/P/0002005018.01.MZZZZZZZ.jpg'

# Popularity Based Recommender System


In [234]:
# Merge the ratings dataset with the books dataset based on the 'ISBN' column to include book details in the ratings data
ratings_with_name = ratings.merge(books,on='ISBN')

In [235]:
ratings_with_name

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,276725,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,https://images.amazon.com/images/P/034545104X....,http://images.amazon.com/images/P/034545104X.0...
1,2313,034545104X,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,https://images.amazon.com/images/P/034545104X....,http://images.amazon.com/images/P/034545104X.0...
2,6543,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,https://images.amazon.com/images/P/034545104X....,http://images.amazon.com/images/P/034545104X.0...
3,8680,034545104X,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,https://images.amazon.com/images/P/034545104X....,http://images.amazon.com/images/P/034545104X.0...
4,10314,034545104X,9,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,https://images.amazon.com/images/P/034545104X....,http://images.amazon.com/images/P/034545104X.0...
...,...,...,...,...,...,...,...,...,...,...
1031131,276688,0517145553,0,Mostly Harmless,Douglas Adams,1995,Random House Value Pub,http://images.amazon.com/images/P/0517145553.0...,https://images.amazon.com/images/P/0517145553....,http://images.amazon.com/images/P/0517145553.0...
1031132,276688,1575660792,7,Gray Matter,Shirley Kennett,1996,Kensington Publishing Corporation,http://images.amazon.com/images/P/1575660792.0...,https://images.amazon.com/images/P/1575660792....,http://images.amazon.com/images/P/1575660792.0...
1031133,276690,0590907301,0,Triplet Trouble and the Class Trip (Triplet Tr...,Debbie Dadey,1997,Apple,http://images.amazon.com/images/P/0590907301.0...,https://images.amazon.com/images/P/0590907301....,http://images.amazon.com/images/P/0590907301.0...
1031134,276704,0679752714,0,A Desert of Pure Feeling (Vintage Contemporaries),Judith Freeman,1997,Vintage Books USA,http://images.amazon.com/images/P/0679752714.0...,https://images.amazon.com/images/P/0679752714....,http://images.amazon.com/images/P/0679752714.0...


In [236]:
ratings_with_name.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1031136 entries, 0 to 1031135
Data columns (total 10 columns):
 #   Column               Non-Null Count    Dtype 
---  ------               --------------    ----- 
 0   User-ID              1031136 non-null  int64 
 1   ISBN                 1031136 non-null  object
 2   Book-Rating          1031136 non-null  int64 
 3   Book-Title           1031136 non-null  object
 4   Book-Author          1031134 non-null  object
 5   Year-Of-Publication  1031136 non-null  object
 6   Publisher            1031134 non-null  object
 7   Image-URL-S          1031136 non-null  object
 8   Image-URL-M          1031136 non-null  object
 9   Image-URL-L          1031132 non-null  object
dtypes: int64(2), object(8)
memory usage: 78.7+ MB


In [237]:
# Group the merged dataset by book title and count the number of ratings for each book
num_rating_df = ratings_with_name.groupby('Book-Title').count()['Book-Rating'].reset_index()

# Rename the column 'Book-Rating' to 'num_ratings' to represent the count of ratings
num_rating_df.rename(columns={'Book-Rating':'num_ratings'},inplace=True)
num_rating_df

,Book-Title,num_ratings
0,A Light in the Storm: The Civil War Diary of ...,4
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,"Ask Lily (Young Women of Faith: Lily Series, ...",1
4,Beyond IBM: Leadership Marketing and Finance ...,1
...,...,...
241066,Ã?Â?lpiraten.,2
241067,Ã?Â?rger mit Produkt X. Roman.,4
241068,Ã?Â?sterlich leben.,1
241069,Ã?Â?stlich der Berge.,3


In [238]:
# Calculate the average rating for each book by grouping the merged dataset by book title
avg_rating_df = ratings_with_name.groupby('Book-Title')['Book-Rating'].mean().reset_index()

# Rename the column 'Book-Rating' to 'num_ratings' to represent the count of ratings
avg_rating_df.rename(columns={'Book-Rating': 'avg_rating'}, inplace=True)
avg_rating_df


,Book-Title,avg_rating
0,A Light in the Storm: The Civil War Diary of ...,2.250000
1,Always Have Popsicles,0.000000
2,Apple Magic (The Collector's series),0.000000
3,"Ask Lily (Young Women of Faith: Lily Series, ...",8.000000
4,Beyond IBM: Leadership Marketing and Finance ...,0.000000
...,...,...
241066,Ã?Â?lpiraten.,0.000000
241067,Ã?Â?rger mit Produkt X. Roman.,5.250000
241068,Ã?Â?sterlich leben.,7.000000
241069,Ã?Â?stlich der Berge.,2.666667


In [239]:
# Merge the DataFrames containing the number of ratings and average ratings for each book based on book title
popular_df = num_rating_df.merge(avg_rating_df,on='Book-Title')

popular_df


,Book-Title,num_ratings,avg_rating
0,A Light in the Storm: The Civil War Diary of ...,4,2.250000
1,Always Have Popsicles,1,0.000000
2,Apple Magic (The Collector's series),1,0.000000
3,"Ask Lily (Young Women of Faith: Lily Series, ...",1,8.000000
4,Beyond IBM: Leadership Marketing and Finance ...,1,0.000000
...,...,...,...
241066,Ã?Â?lpiraten.,2,0.000000
241067,Ã?Â?rger mit Produkt X. Roman.,4,5.250000
241068,Ã?Â?sterlich leben.,1,7.000000
241069,Ã?Â?stlich der Berge.,3,2.666667


In [240]:
popular_df = popular_df[popular_df['num_ratings']>=250].sort_values('avg_rating',ascending=False).head(50)
popular_df

,Book-Title,num_ratings,avg_rating
80434,Harry Potter and the Prisoner of Azkaban (Book 3),428,5.852804
80422,Harry Potter and the Goblet of Fire (Book 4),387,5.824289
80441,Harry Potter and the Sorcerer's Stone (Book 1),278,5.737410
80426,Harry Potter and the Order of the Phoenix (Boo...,347,5.501441
80414,Harry Potter and the Chamber of Secrets (Book 2),556,5.183453
191612,The Hobbit : The Enchanting Prelude to The Lor...,281,5.007117
187377,The Fellowship of the Ring (The Lord of the Ri...,368,4.948370
80445,Harry Potter and the Sorcerer's Stone (Harry P...,575,4.895652
211384,"The Two Towers (The Lord of the Rings, Part 2)",260,4.880769
219741,To Kill a Mockingbird,510,4.700000


In [241]:
# Merge the DataFrame containing popular books with the original books dataset based on book title,
# drop duplicate entries based on book title, and select specific columns for the final DataFrame

popular_df = popular_df.merge(books,on='Book-Title').drop_duplicates('Book-Title')[['Book-Title','Book-Author','Year-Of-Publication','Image-URL-M','num_ratings','avg_rating']]
popular_df

,Book-Title,Book-Author,Year-Of-Publication,Image-URL-M,num_ratings,avg_rating
0,Harry Potter and the Prisoner of Azkaban (Book 3),J. K. Rowling,1999,https://images.amazon.com/images/P/0439136350....,428,5.852804
3,Harry Potter and the Goblet of Fire (Book 4),J. K. Rowling,2000,https://images.amazon.com/images/P/0439139597....,387,5.824289
5,Harry Potter and the Sorcerer's Stone (Book 1),J. K. Rowling,1998,https://images.amazon.com/images/P/0590353403....,278,5.737410
9,Harry Potter and the Order of the Phoenix (Boo...,J. K. Rowling,2003,https://images.amazon.com/images/P/043935806X....,347,5.501441
13,Harry Potter and the Chamber of Secrets (Book 2),J. K. Rowling,2000,https://images.amazon.com/images/P/0439064872....,556,5.183453
16,The Hobbit : The Enchanting Prelude to The Lor...,J.R.R. TOLKIEN,1986,https://images.amazon.com/images/P/0345339681....,281,5.007117
17,The Fellowship of the Ring (The Lord of the Ri...,J.R.R. TOLKIEN,1986,https://images.amazon.com/images/P/0345339703....,368,4.948370
26,Harry Potter and the Sorcerer's Stone (Harry P...,J. K. Rowling,1999,https://images.amazon.com/images/P/059035342X....,575,4.895652
28,"The Two Towers (The Lord of the Rings, Part 2)",J.R.R. TOLKIEN,1986,https://images.amazon.com/images/P/0345339711....,260,4.880769
39,To Kill a Mockingbird,Harper Lee,1988,https://images.amazon.com/images/P/0446310786....,510,4.700000


# Collaborative Filtering Based Recommender System


In [242]:
# Group the ratings_with_name dataframe by User-ID and count the number of ratings for each user
# Check if the count is greater than 200 to identify users with substantial rating activity
x = ratings_with_name.groupby('User-ID').count()['Book-Rating'] > 200

# Extract the indices of users who have rated more than 200 books
achhe_users = x[x].index
achhe_users

Index([   254,   2276,   2766,   2977,   3363,   4017,   4385,   6251,   6323,
         6543,
       ...
       271705, 273979, 274004, 274061, 274301, 274308, 275970, 277427, 277639,
       278418],
      dtype='int64', name='User-ID', length=811)

In [243]:
# Filter the ratings_with_name dataframe to include only ratings from users with substantial rating activity
filtered_rating = ratings_with_name[ratings_with_name['User-ID'].isin(achhe_users)]
filtered_rating

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
2,6543,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,https://images.amazon.com/images/P/034545104X....,http://images.amazon.com/images/P/034545104X.0...
5,23768,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,https://images.amazon.com/images/P/034545104X....,http://images.amazon.com/images/P/034545104X.0...
7,28523,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,https://images.amazon.com/images/P/034545104X....,http://images.amazon.com/images/P/034545104X.0...
15,77940,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,https://images.amazon.com/images/P/034545104X....,http://images.amazon.com/images/P/034545104X.0...
16,81977,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,https://images.amazon.com/images/P/034545104X....,http://images.amazon.com/images/P/034545104X.0...
...,...,...,...,...,...,...,...,...,...,...
1030883,275970,1880837927,0,The Theology of the Hammer,Millard Fuller,1994,Smyth &amp; Helwys Publishing,http://images.amazon.com/images/P/1880837927.0...,https://images.amazon.com/images/P/1880837927....,http://images.amazon.com/images/P/1880837927.0...
1030884,275970,188717897X,0,The Ordeal of Integration: Progress and Resent...,Orlando Patterson,1998,Civitas Book Publisher,http://images.amazon.com/images/P/188717897X.0...,https://images.amazon.com/images/P/188717897X....,http://images.amazon.com/images/P/188717897X.0...
1030885,275970,1888889047,0,Pushcart's Complete Rotten Reviews &amp; Rejec...,Bill Henderson,1998,Pushcart Press,http://images.amazon.com/images/P/1888889047.0...,https://images.amazon.com/images/P/1888889047....,http://images.amazon.com/images/P/1888889047.0...
1030886,275970,1931868123,0,There's a Porcupine in My Outhouse: Misadventu...,Mike Tougias,2002,Capital Books (VA),http://images.amazon.com/images/P/1931868123.0...,https://images.amazon.com/images/P/1931868123....,http://images.amazon.com/images/P/1931868123.0...


In [244]:
# Group the filtered ratings dataframe by book title and count the number of ratings for each book
# Check if the count is greater than or equal to 50 to identify books with significant no of ratings
y = filtered_rating.groupby('Book-Title').count()['Book-Rating'] >= 50

# Extract the indices of books that have received at least 50 ratings
famous_books = y[y].index
famous_books

Index(['1984', '1st to Die: A Novel', '2nd Chance', '4 Blondes',
       'A Bend in the Road', 'A Case of Need',
       'A Child Called \It\": One Child's Courage to Survive"',
       'A Civil Action', 'A Day Late and a Dollar Short', 'A Fine Balance',
       ...
       'Winter Solstice', 'Wish You Well', 'Without Remorse',
       'Wizard and Glass (The Dark Tower, Book 4)', 'Wuthering Heights',
       'Year of Wonders', 'You Belong To Me',
       'Zen and the Art of Motorcycle Maintenance: An Inquiry into Values',
       'Zoya', '\O\" Is for Outlaw"'],
      dtype='object', name='Book-Title', length=706)

In [245]:
# Filter the filtered ratings dataset to include only ratings for books with significant no. of ratings
final_ratings = filtered_rating[filtered_rating['Book-Title'].isin(famous_books)]

# Display the final ratings dataset containing ratings and other details for books having significant no. of ratings
final_ratings

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
63,278418,0446520802,0,The Notebook,Nicholas Sparks,1996,Warner Books,http://images.amazon.com/images/P/0446520802.0...,https://images.amazon.com/images/P/0446520802....,http://images.amazon.com/images/P/0446520802.0...
65,3363,0446520802,0,The Notebook,Nicholas Sparks,1996,Warner Books,http://images.amazon.com/images/P/0446520802.0...,https://images.amazon.com/images/P/0446520802....,http://images.amazon.com/images/P/0446520802.0...
66,7158,0446520802,10,The Notebook,Nicholas Sparks,1996,Warner Books,http://images.amazon.com/images/P/0446520802.0...,https://images.amazon.com/images/P/0446520802....,http://images.amazon.com/images/P/0446520802.0...
69,11676,0446520802,10,The Notebook,Nicholas Sparks,1996,Warner Books,http://images.amazon.com/images/P/0446520802.0...,https://images.amazon.com/images/P/0446520802....,http://images.amazon.com/images/P/0446520802.0...
74,23768,0446520802,6,The Notebook,Nicholas Sparks,1996,Warner Books,http://images.amazon.com/images/P/0446520802.0...,https://images.amazon.com/images/P/0446520802....,http://images.amazon.com/images/P/0446520802.0...
...,...,...,...,...,...,...,...,...,...,...
1026724,266865,0531001725,10,The Catcher in the Rye,Jerome David Salinger,1973,Scholastic Library Pub,http://images.amazon.com/images/P/0531001725.0...,https://images.amazon.com/images/P/0531001725....,http://images.amazon.com/images/P/0531001725.0...
1027923,269566,0670809381,0,Echoes,Maeve Binchy,1986,Penguin USA,http://images.amazon.com/images/P/0670809381.0...,https://images.amazon.com/images/P/0670809381....,http://images.amazon.com/images/P/0670809381.0...
1028777,271284,0440910927,0,The Rainmaker,John Grisham,1995,Island,http://images.amazon.com/images/P/0440910927.0...,https://images.amazon.com/images/P/0440910927....,http://images.amazon.com/images/P/0440910927.0...
1029070,271705,B0001PIOX4,0,Fahrenheit 451,Ray Bradbury,1993,Simon &amp; Schuster,http://images.amazon.com/images/P/B0001PIOX4.0...,https://images.amazon.com/images/P/B0001PIOX4....,http://images.amazon.com/images/P/B0001PIOX4.0...


#### Vectorization of Books having Users as the elements of vector in higher dimensional space

In [247]:
# Create a pivot table from the final ratings dataset with book titles as rows, User-IDs as columns,
# and Book-Ratings as values to represent user-book interactions
pivot_table = final_ratings.pivot_table(index='Book-Title', columns='User-ID', values='Book-Rating')

# Display the pivot table representing user-book interactions
pivot_table

User-ID,254,2276,2766,2977,3363,4017,4385,6251,6323,6543,...,271705,273979,274004,274061,274301,274308,275970,277427,277639,278418
Book-Title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,10.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN
1st to Die: A Novel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2nd Chance,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN
4 Blondes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A Bend in the Road,0.0,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Year of Wonders,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,9.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN
You Belong To Me,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN


In [248]:
# Replace missing values (NaNs) in the pivot table with zeros to represent no rating given by a user for a particular book
pivot_table.fillna(0, inplace=True)

In [249]:
# Display the pivot table representing user-book interactions as the a vector in higher dimensional spcae.
# Means Each Rows here a vector no of columns dimensional space
pivot_table

User-ID,254,2276,2766,2977,3363,4017,4385,6251,6323,6543,...,271705,273979,274004,274061,274301,274308,275970,277427,277639,278418
Book-Title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Bend in the Road,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Year of Wonders,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
You Belong To Me,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


##### Here in above pivot table, we Have 706 vectors in the 810 dimensional space.

### Computation of Cosine Similarity (Collaborative Filtering)

In [250]:
from sklearn.metrics.pairwise import cosine_similarity

In [251]:
# Calculate cosine similarity scores between books based on user-book interactions represented by the pivot table
similarity_scores = cosine_similarity(pivot_table)
similarity_scores

array([[1.        , 0.10255025, 0.01220856, ..., 0.12110367, 0.07347567,
        0.04316046],
       [0.10255025, 1.        , 0.2364573 , ..., 0.07446129, 0.16773875,
        0.14263397],
       [0.01220856, 0.2364573 , 1.        , ..., 0.04558758, 0.04938579,
        0.10796119],
       ...,
       [0.12110367, 0.07446129, 0.04558758, ..., 1.        , 0.07085128,
        0.0196177 ],
       [0.07347567, 0.16773875, 0.04938579, ..., 0.07085128, 1.        ,
        0.10602962],
       [0.04316046, 0.14263397, 0.10796119, ..., 0.0196177 , 0.10602962,
        1.        ]])

In [252]:
similarity_scores.shape

(706, 706)

#### Building a Function which recommends

In [253]:
# Find the index position of the book say "Animal Farm" in the pivot table
np.where(pivot_table.index=='Animal Farm')

(array([47], dtype=int64),)

In [170]:
index = np.where(pivot_table.index=='Animal Farm')[0][0]


In [171]:
sorted(similarity_scores[index], reverse =True)

[0.9999999999999998,
 0.2702651417103732,
 0.2416404696056939,
 0.2362570952024308,
 0.21897040971615078,
 0.20936355536567294,
 0.2039361513890485,
 0.2029392104193419,
 0.19936781793629338,
 0.1934411223332458,
 0.1933680785174527,
 0.1853698575738056,
 0.18432987234650272,
 0.1828523777485272,
 0.1808593793358965,
 0.18033685629548887,
 0.17857624365011393,
 0.17810293434375038,
 0.17574683842508246,
 0.17550919073101653,
 0.17524780797774,
 0.17285553764932207,
 0.17139647009659337,
 0.1687005636608251,
 0.16727198067474489,
 0.16082459779343755,
 0.16077267549550414,
 0.15870304913591407,
 0.15866348262730293,
 0.1565007975327465,
 0.15632476361389702,
 0.1548168121432704,
 0.1526938495910763,
 0.15260868572873312,
 0.15109390524068056,
 0.1505511478256334,
 0.15009211603421194,
 0.1486861806511982,
 0.14731534531412904,
 0.1472959089266509,
 0.1470104920254031,
 0.14453100696863436,
 0.14436961311545574,
 0.1441287267186271,
 0.1432521990496692,
 0.1426711909607604,
 0.1412494068

In [254]:
# Sort the enumerated cosine similarity scores for the book "Animal Farm" in descending order
# without enumerate, we we only have similarity_score and we lost the index information to fetch boock information
sorted(list(enumerate(similarity_scores[index])),key=lambda x:x[1],reverse=True)

[(47, 0.9999999999999998),
 (0, 0.2702651417103732),
 (45, 0.2416404696056939),
 (324, 0.2362570952024308),
 (422, 0.21897040971615078),
 (92, 0.20936355536567294),
 (344, 0.2039361513890485),
 (309, 0.2029392104193419),
 (500, 0.19936781793629338),
 (82, 0.1934411223332458),
 (695, 0.1933680785174527),
 (86, 0.1853698575738056),
 (648, 0.18432987234650272),
 (650, 0.1828523777485272),
 (238, 0.1808593793358965),
 (389, 0.18033685629548887),
 (304, 0.17857624365011393),
 (175, 0.17810293434375038),
 (40, 0.17574683842508246),
 (178, 0.17550919073101653),
 (432, 0.17524780797774),
 (610, 0.17285553764932207),
 (19, 0.17139647009659337),
 (192, 0.1687005636608251),
 (507, 0.16727198067474489),
 (156, 0.16082459779343755),
 (202, 0.16077267549550414),
 (691, 0.15870304913591407),
 (5, 0.15866348262730293),
 (312, 0.1565007975327465),
 (667, 0.15632476361389702),
 (626, 0.1548168121432704),
 (318, 0.1526938495910763),
 (384, 0.15260868572873312),
 (550, 0.15109390524068056),
 (258, 0.15055

In [255]:
def recommend(book_name):
    """
    Recommends similar books based on the input book name.

    Parameters:
    - book_name (str): The name of the book for which recommendations are sought.

    Returns:
    - data (list): A list containing information about recommended books, including title, author, publication year, and image URL.
    """

    # Find the index position of the input book in the pivot table
    index = np.where(pivot_table.index == book_name)[0][0]

    # Find similar items based on cosine similarity scores, excluding the input book itself
    similar_items = sorted(list(enumerate(similarity_scores[index])), key=lambda x: x[1], reverse=True)[1:5]

    # Prepare data for recommended books
    data = []
    for i in similar_items:
        item = []
        # Extract information about recommended books from the books dataset
        temp_df = books[books['Book-Title'] == pivot_table.index[i[0]]]
        item.extend(list(temp_df.drop_duplicates('Book-Title')['Book-Title'].values))
        item.extend(list(temp_df.drop_duplicates('Book-Title')['Book-Author'].values))
        item.extend(list(temp_df.drop_duplicates('Book-Title')['Year-Of-Publication'].values))
        item.extend(list(temp_df.drop_duplicates('Book-Title')['Image-URL-M'].values))
        
        data.append(item)
    
    return data

In [256]:
# Checking and verifying the recommendations for a Book
recommend('Animal Farm')

[['1984',
  'George Orwell',
  1990,
  'https://images.amazon.com/images/P/0451524934.01.MZZZZZZZ.jpg'],
 ['Angus, Thongs and Full-Frontal Snogging: Confessions of Georgia Nicolson',
  'Louise Rennison',
  2001,
  'https://images.amazon.com/images/P/0064472272.01.MZZZZZZZ.jpg'],
 ['Midnight',
  'Dean R. Koontz',
  1995,
  'https://images.amazon.com/images/P/0425118703.01.MZZZZZZZ.jpg'],
 ['Second Nature',
  'Alice Hoffman',
  1994,
  'https://images.amazon.com/images/P/0399139087.01.MZZZZZZZ.jpg']]

In [258]:
# List of books there in the pivot table
list(pivot_table.index)

['1984',
 '1st to Die: A Novel',
 '2nd Chance',
 '4 Blondes',
 'A Bend in the Road',
 'A Case of Need',
 'A Child Called \\It\\": One Child\'s Courage to Survive"',
 'A Civil Action',
 'A Day Late and a Dollar Short',
 'A Fine Balance',
 'A Great Deliverance',
 'A Heartbreaking Work of Staggering Genius',
 'A Is for Alibi (Kinsey Millhone Mysteries (Paperback))',
 'A Lesson Before Dying (Vintage Contemporaries (Paperback))',
 'A Man Named Dave: A Story of Triumph and Forgiveness',
 'A Man in Full',
 'A Map of the World',
 'A Painted House',
 'A Patchwork Planet',
 'A Prayer for Owen Meany',
 'A Thin Dark Line (Mysteries &amp; Horror)',
 "A Thousand Acres (Ballantine Reader's Circle)",
 'A Time to Kill',
 "A Virtuous Woman (Oprah's Book Club (Paperback))",
 'A Walk to Remember',
 'A Widow for One Year',
 'A Wrinkle In Time',
 'A Wrinkle in Time',
 'A Year in Provence',
 "ANGELA'S ASHES",
 'About a Boy',
 'Absolute Power',
 'Acceptable Risk',
 'Accident',
 'Airframe',
 'All Around the To

In [176]:
# Replace 'http' with 'https' in the 'Image-URL-M' column
books['Image-URL-M'] = books['Image-URL-M'].str.replace('http://', 'https://')

# Verify the changes
books['Image-URL-M'].head()

0    https://images.amazon.com/images/P/0195153448....
1    https://images.amazon.com/images/P/0002005018....
2    https://images.amazon.com/images/P/0060973129....
3    https://images.amazon.com/images/P/0374157065....
4    https://images.amazon.com/images/P/0393045218....
Name: Image-URL-M, dtype: object

#### Serialize (Storing) the impoartant dataframes(informations) For buliding WebApp using pickle


In [177]:
import pickle

In [260]:
# Remove duplicate entries in the books DataFrame based on the 'Book-Title' column
books = books.drop_duplicates('Book-Title')

books

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,https://images.amazon.com/images/P/0195153448....,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,https://images.amazon.com/images/P/0002005018....,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,https://images.amazon.com/images/P/0060973129....,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,https://images.amazon.com/images/P/0374157065....,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,https://images.amazon.com/images/P/0393045218....,http://images.amazon.com/images/P/0393045218.0...
...,...,...,...,...,...,...,...,...
271354,0449906736,Flashpoints: Promise and Peril in a New World,Robin Wright,1993,Ballantine Books,http://images.amazon.com/images/P/0449906736.0...,https://images.amazon.com/images/P/0449906736....,http://images.amazon.com/images/P/0449906736.0...
271356,0525447644,From One to One Hundred,Teri Sloat,1991,Dutton Books,http://images.amazon.com/images/P/0525447644.0...,https://images.amazon.com/images/P/0525447644....,http://images.amazon.com/images/P/0525447644.0...
271357,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker,2004,HarperSanFrancisco,http://images.amazon.com/images/P/006008667X.0...,https://images.amazon.com/images/P/006008667X....,http://images.amazon.com/images/P/006008667X.0...
271358,0192126040,Republic (World's Classics),Plato,1996,Oxford University Press,http://images.amazon.com/images/P/0192126040.0...,https://images.amazon.com/images/P/0192126040....,http://images.amazon.com/images/P/0192126040.0...


In [261]:
# Checking the Count of duplicated entries in the books DataFrame based on the 'Book-Title' column
books.duplicated('Book-Title').sum()

0

In [264]:
# Serializing (Store) the popular_df, pivot table, books DataFrame, and similarity scores using pickle

# Dump the popular_df DataFrame into a binary file named 'popular.pkl'
pickle.dump(popular_df,open('popular.pkl','wb'))

# Dump the pivot table into a binary file named 'pt.pkl'
pickle.dump(pivot_table, open('pivot_table.pkl', 'wb'))

# Dump the books DataFrame into a binary file named 'books.pkl'
pickle.dump(books, open('books.pkl', 'wb'))

# Dump the similarity scores into a binary file named 'similarity_scores.pkl'
pickle.dump(similarity_scores, open('similarity_scores.pkl', 'wb'))